In [20]:
import numpy as np
import openai
import pandas as pd
import tiktoken
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

# 1. Grab docs and parse them into a table

In [29]:
sitemap = 'https://docs.getdbt.com/sitemap.xml'
sitemap_res = requests.get(sitemap)
sitemap_tree = ET.fromstring(sitemap_res.content)

urls = []
for url in sitemap_tree.findall("{http://www.sitemaps.org/schemas/sitemap/0.9}url"):
    loc = url.find("{http://www.sitemaps.org/schemas/sitemap/0.9}loc")
    urls.append(loc.text)

In [71]:
table = []

for url in urls:
    response = requests.get(url)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    current_heading = ''
    current_content = ''
    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'p']):
        if tag.name.startswith('h'):
            # If we encounter a new heading, add the previous content to the table
            if current_heading and current_content:
                row = {}
                row['title'] = soup.title.string.strip() if soup.title else ''
                row['heading'] = current_heading
                row['content'] = current_content
                row['tokens'] = len(current_content.split())
                table.append(row)
            # Update the current heading and reset the current content
            current_heading = tag.text.strip()
            current_content = ''
        else:
            # Add the text to the current content for this heading
            current_content += tag.text.strip()

    # Add the last row to the table
    if current_heading and current_content:
        row = {}
        row['title'] = soup.title.string.strip() if soup.title else ''
        row['heading'] = current_heading
        row['content'] = current_content
        row['tokens'] = len(current_content.split())
        table.append(row)
    
    
df = pd.DataFrame(table)
df = df.set_index(["title", "heading"])
df = df.drop_duplicates()

We preprocess the document sections by creating an embedding vector for each section. An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar are their contents. See the [documentation on OpenAI embeddings](https://beta.openai.com/docs/guides/embeddings) for more information.

This indexing stage can be executed offline and only runs once to precompute the indexes for the dataset so that each piece of content can be retrieved later. Since this is a small example, we will store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/), [Weaviate](https://github.com/semi-technologies/weaviate) or [Qdrant](https://qdrant.tech) to power the search.

In [68]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [69]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [72]:
document_embeddings = compute_doc_embeddings(df)

So we have split our document library into sections, and encoded them by creating embedding vectors that represent each chunk. Next we will use these embeddings to answer our users' questions.

# 2) Find the most similar document embeddings to the question embedding

At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/), [Weaviate](https://github.com/semi-technologies/weaviate) or [Qdrant](https://qdrant.tech) to power the search.

In [73]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

# 3) Add the most relevant document sections to the query prompt

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [74]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [110]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.
        document_section = df.loc[section_index]
        document_section = document_section.iloc[0]

        # We've added `first` here because we seem to still be dealing with non-unique multi-indices
        chosen_sections_len += document_section.tokens + separator_len
        
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [111]:
construct_prompt('What is dbt?', document_embeddings, df)

Selected 12 document sections:
('Getting help | dbt Developer Hub', 'dbt Training\u200b')
('Overview | dbt Developer Hub', 'How do I get started?')
('Overview | dbt Developer Hub', 'What is dbt?')
('debug | dbt Developer Hub', 'Usage\u200b')
('What is dbt? | dbt Developer Hub', 'About dbt\u200b')
('dbt Developer Hub', 'Docs')
('Teradata setup | dbt Developer Hub', 'Commands\u200b')
('Updating our permissioning guidelines: grants as configs in dbt Core v1.2 | dbt Developer Blog', 'The solution then\u200b')
('dbt Developer Hub', 'Supported data platforms')
('What is dbt? | dbt Developer Hub', 'dbt Core\u200b')
('dbt Developer Blog', 'dbt Developer Blog')
('dbt Developer Hub', 'What is dbt?')


/var/folders/qk/p4_th7xd6z91xpbf0hgtbk3w0000gp/T/ipykernel_36968/2959345404.py:13: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


'Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don\'t know."\n\nContext:\n\n* If you want to receive dbt training, check out our dbt Learn program.\n* dbt is open source and completely free to download and use. See our Getting Started guide for more.\n* dbt is a productivity tool that helps analysts get more done and produce higher quality results.Analysts commonly spend 50-80% of their time modeling raw data—cleaning, reshaping, and applying fundamental business logic to it. dbt empowers analysts to do this work better and faster.dbt\'s primary interface is its CLI. Using dbt is a combination of editing code in a text editor and running that code using dbt from the command line using dbt [command] [options].\n* When dbt hits the debug() line, you\'ll see something like:\n* dbt is a transformation workflow that helps you get more work done while producing higher quality results. You can use dbt

We have now obtained the document sections that are most relevant to the question. As a final step, let's put it all together to get an answer to the question.

# 4) Answer the user's question based on the context.

Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.

In [112]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [113]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [123]:
question = "How do I write an incremental model in dbt? Can you show me example code?"
answer_query_with_context(question, df, document_embeddings)

/var/folders/qk/p4_th7xd6z91xpbf0hgtbk3w0000gp/T/ipykernel_36968/2959345404.py:13: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index]


Selected 6 document sections:
('Incremental models | dbt Developer Hub', 'Overview\u200b')
('Materializations | dbt Developer Hub', 'Incremental\u200b')
('Incremental models in-depth | dbt Developer Hub', 'Configuring incremental models\u200b')
('Incremental models in-depth | dbt Developer Hub', 'Incremental models in-depth')
('Incremental models | dbt Developer Hub', 'Configuring incremental strategy\u200b')
('Incremental models in-depth | dbt Developer Hub', 'Writing incremental logic\u200b')


"You can write an incremental model in dbt using the {{ this }} keyword. An example code block would look like this: \n\n`SELECT * FROM {{ source('my_table') }} WHERE {{ this.created_at > ref('last_run') }}`"